In [1]:
import paramiko
import pandas as pd
from io import BytesIO 
import oracledb
import pandas as pd
from sqlalchemy import create_engine
import os

c:\Users\MR38804\AppData\Local\anaconda3\envs\Py64\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
c:\Users\MR38804\AppData\Local\anaconda3\envs\Py64\Lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.Blowfish and will be removed from this module in 45.0.0.
  "class": algorithms.Blowfish,
c:\Users\MR38804\AppData\Local\anaconda3\envs\Py64\Lib\site-packages\paramiko\transport.py:243: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [2]:
# # credentials DBA
# username = os.environ['DBA_USR']
# password = os.environ['DBA_PW']
# dsn = '10.220.50.121:1661/ANALYTIC'

In [ ]:
# credentials EDWB
username = os.environ['UID']
password = os.environ['EDWB_PW']
dsn = '172.16.2.28:1521/brgprod'

In [4]:
# create connection
connection = oracledb.connect(user=username, password=password, dsn=dsn)

In [5]:
# create engine
engine = create_engine('oracle+oracledb://', creator=lambda: connection)

In [6]:
# dfs = []
# # 20240930 2409
# # ym360 = [2408, 2404,2402, 2311,2310,2308,2307]
# # dtML= [20240830,20240430,20240229,20231130,
# # 20231031,20230831,20230731]

# # new pull
# # 2305,2304,2303,2302,2301,2212,2211,2210,2209,2208,2207,2206,2205,2204,2203,2202,2201
# ym360 = [2303,
#          2302,2301,2212,2211,2210,2209,
#          2208,2207,2206,2205,2204,2203]

# # open and read the file as a single buffer
# for i in range(len(ym360)):
#     ym = ym360[i]
#     query = f'''SELECT a.BASE_DT, a.APPL_ID, a.BR_CD, a.GCIF_NO, a.CIF_NO,
# a.NOTE_NO, a.PRD_TP, a.STATUS, a.ORG_LMT_AMT, a.BAL, a.ORIG_INT_RT,
# a.CURR_INT_RT, a.TENOR , a.DAYS_REMAIN, a.LTV, a.RESTRUCT_CD, 
# a.PROJ_VAL, a.CLCT_RTNG_FCL, a.NPL_YN,
# b.AGE, b.LAST_ACTIVE_DATE, b.OPENDATE AS OPEN_DATE, b.EMPLOYMENT_TYPE, 
# b.ANNUAL_INCOME, b.ANNUAL_SALES, b.CUST_TP, b.CASHLOAN_PLAFOND, b.CASHLOAN_OS,
# b.CC,b.CC_PLAFOND,b.PL_PLAFOND,b.PLOAN_BAL, b.PROD_HOLDING,
# b.LOB as SEGMENT,
# b.MARITAL_STATUS, b.TRB, b.ACTIVENESS, b.FUNDING_BAL,b.FUNDING_AVG, b.DBTRX,
# b.DBTRXAMT, b.CRTRX, b.CRTRXAMT
# FROM PDA.MASTER_LOAN a
# LEFT JOIN PDA.CUSTOMER_PORTFOLIO_{ym} b
# ON a.GCIF_NO = b.GCIF_NO
# WHERE a.BASE_DT = (SELECT MAX(BASE_DT) FROM PDA.MASTER_LOAN WHERE BASE_DT LIKE '20{ym}%')
# AND a.PRD_TP IN ('133','173','603','613','673','776','763','773','782','783')'''
#     df = pd.read_sql(query, engine)
#     df.columns = map(str.upper, df.columns)
#     df.to_csv(f'cust_loan_{ym}.dsv', sep='|', index=False)
#     dfs.append(df)
#     print()

In [7]:
dfs = []
# 20240930 2409
# ym360 = [2408, 2404,2402, 2311,2310,2308,2307]
# dtML= [20240830,20240430,20240229,20231130,
# 20231031,20230831,20230731]

# new pull
# 2305,2304,2303,2302,2301,2212,2211,2210,2209,2208,2207,2206,2205,2204,2203,2202,2201
ym360 = [230331,
         230228,230131,221230,221130,221031,220930,
         220831,220729,220630,220531,220428,220331]

# open and read the file as a single buffer
for i in range(len(ym360)):
    dt = ym360[i]
    ym = str(ym360[i])[:6]
    query = f'''SELECT CUST_NO, NOTE_NO,STATUS,NOTE_DATE,ORIG_MATURITY_DATE, PRIN_BAL,ORIG_INT_RATE, CUR_INT_RATE,NXT_CHG_DATE
FROM BRGPROD.V_CLFS{ym}'''
    df = pd.read_sql(query, engine)
    df.columns = map(str.upper, df.columns)
    df.to_csv(f'CLFS_{ym}.csv', sep=',', index=False)
    dfs.append(df)
    print()

In [9]:
dfs[0]

,CUST_NO,NOTE_NO,STATUS,NOTE_DATE,ORIG_MATURITY_DATE,PRIN_BAL,ORIG_INT_RATE,CUR_INT_RATE,NXT_CHG_DATE
0,0001772032,39393304,C,2022-09-26,2023-03-26,0.000000e+00,0.0800,0.0800,2022-09-26
1,0001772032,39393304,C,2022-09-28,2023-03-28,0.000000e+00,0.0800,0.0800,2022-09-28
2,0002828723,39888801,A,2023-01-10,2024-01-04,0.000000e+00,0.0950,0.0950,2023-01-10
3,0003959277,16158302,A,2014-06-06,2024-06-06,1.223958e+08,0.1275,0.1275,2024-06-06
4,0001772032,39393304,A,2023-02-22,2023-08-22,3.000000e+10,0.0800,0.0800,2023-02-22
...,...,...,...,...,...,...,...,...,...
63896,0011294560,45555402,A,2021-03-24,2023-12-18,0.000000e+00,0.0000,0.0000,2025-12-18
63897,0011294560,45555402,A,2021-03-24,2025-02-18,0.000000e+00,0.0000,0.0000,2025-12-18
63898,0011294560,45555402,A,2021-03-24,2025-12-18,0.000000e+00,0.0000,0.0000,2024-12-18
63899,0011294560,45555402,A,2021-12-27,2025-12-18,0.000000e+00,0.0000,0.0000,2025-12-18


In [7]:
# ym = 2303
# dt = 20230331

In [8]:
# print(query)